# <font color="#418FDE" size="6.5" uppercase>**Context Management and Cleanup: contextlib-related Patterns and with Protocol**</font>

>Last update: 20251207.
    
By the end of this Lecture, you will be able to:
- Explain how the with statement uses the context management protocol with built-in types in Python 3.12. 
- Use built-in types such as files and locks within with blocks to guarantee cleanup. 
- Design simple context-managed utilities that integrate cleanly with built-in behavior. 


## **1. The with Statement and the Context Management Protocol**

### **1.1. Conceptual Model of __enter__ and __exit__ in Context Managers**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_01_01.jpg?v=1765169672" width="250">



>* with calls enter to set up resources
>* block runs, then exit handles cleanup reliably

>* Enter sets up resources and returns interface
>* Exit finalizes work and restores original state

>* exit always runs, even when exceptions occur
>* exit can clean up, handle, or suppress errors



In [ ]:
#@title Python Code - Conceptual Model of __enter__ and __exit__ in Context Managers

# Demonstrate basic context manager enter and exit behavior.
# Show how with calls setup and cleanup methods automatically.
# Illustrate how exceptions still trigger cleanup and optional suppression.

class SimpleContext:
    def __enter__(self):
        print("__enter__ called, setting up resource.")
        return "hammer tool ready"

    def __exit__(self, exc_type, exc_value, exc_tb):
        print("__exit__ called, cleaning up resource.")
        if exc_type is None:
            print("Block finished normally, nothing exceptional happened.")
            return False

        print("Block raised exception type:", exc_type.__name__)
        print("Exception message was:", exc_value)
        print("Exception will be suppressed here.")
        return True

print("Starting first with block demonstration.")
with SimpleContext() as tool:
    print("Inside with block, using:", tool)
    print("Work finished successfully inside block.")

print("Starting second with block demonstration.")
with SimpleContext() as tool:
    print("Inside with block, using:", tool)
    raise ValueError("Simulated problem while using tool.")




### **1.2. Guaranteed Cleanup on Success or Failure**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_01_02.jpg?v=1765169725" width="250">



>* Context managers always run their cleanup code
>* Prevents resource leaks and supports reliable software

>* Resource leaks can crash long-running services
>* Context managers always release resources on exit

>* Define clear lifetimes for each acquired resource
>* Predictable cleanup aids safety, compliance, and auditing



In [ ]:
#@title Python Code - Guaranteed Cleanup on Success or Failure

# Demonstrate guaranteed cleanup when code inside with block succeeds or fails.
# Show that file closes even when an exception interrupts normal processing.
# Compare manual open-close pattern with automatic with-statement cleanup behavior.

from pathlib import Path
from contextlib import contextmanager

@contextmanager
def demo_resource(name_label):
    print(f"Acquiring resource named {name_label} now.")
    try:
        yield f"Resource({name_label})"
    finally:
        print(f"Cleaning up resource named {name_label} now.")


def process_with_optional_error(should_fail_flag):
    print(f"Starting processing, failure flag equals {should_fail_flag}.")
    with demo_resource("important_file") as resource_label:
        print(f"Using resource inside with block: {resource_label}.")
        if should_fail_flag:
            print("About to raise simulated error now.")
            raise RuntimeError("Simulated processing failure occurred.")
        print("Processing finished successfully without errors.")


for flag_value in (False, True):
    try:
        process_with_optional_error(flag_value)
    except RuntimeError as error_instance:
        print(f"Caught error outside with block: {error_instance}.")
    print("Run finished, resource already cleaned up above.\n")




### **1.3. Managing Multiple Resources with Nested with Blocks**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_01_03.jpg?v=1765169784" width="250">



>* Nested with blocks manage multiple resources cleanly
>* Enter acquires each resource, exit reliably cleans up

>* Nested with blocks simplify error-path cleanup
>* Resources are released in reverse acquisition order

>* Nested with blocks coordinate many related resources
>* They guarantee predictable cleanup and improve code readability



In [ ]:
#@title Python Code - Managing Multiple Resources with Nested with Blocks

# Demonstrate nested with blocks managing multiple resources safely.
# Show automatic cleanup when everything works normally.
# Show automatic cleanup when an error happens inside.

from contextlib import contextmanager
import threading

@contextmanager
def fake_lock(name):
    print(f"Acquiring lock named {name} now.")
    lock = threading.Lock()
    lock.acquire()
    try:
        yield lock
    finally:
        lock.release()
        print(f"Releasing lock named {name} now.")


@contextmanager
def demo_file(path, mode, label):
    print(f"Opening file labeled {label} at {path}.")
    f = open(path, mode, encoding="utf-8")
    try:
        yield f
    finally:
        f.close()
        print(f"Closing file labeled {label} at {path}.")


print("Starting normal nested with example now.")
with demo_file("input_demo.txt", "w", "input") as input_file:
    input_file.write("Line one inches.\n")
    with demo_file("output_demo.txt", "w", "output") as output_file:
        with fake_lock("demo_lock") as lock:
            output_file.write("Copied line one inches.\n")
            print("Inside deepest block doing safe work.")


print("\nStarting error nested with example now.")
try:
    with demo_file("input_demo_two.txt", "w", "input_two") as input_two:
        input_two.write("Second example inches.\n")
        with demo_file("output_demo_two.txt", "w", "output_two") as output_two:
            with fake_lock("error_lock") as error_lock:
                print("About to raise error inside deepest block.")
                raise ValueError("Simulated processing failure inches.")
except ValueError as exc:
    print(f"Caught ValueError outside with blocks: {exc}.")




## **2. Using Built-in Context Managers for Safe Cleanup**

### **2.1. Safely Managing Files with open() and with Blocks**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_02_01.jpg?v=1765169844" width="250">



>* with blocks auto-close files, even on errors
>* prevents resource leaks and keeps long programs stable

>* Context managers ensure files always close correctly
>* You focus on data logic, not cleanup

>* Context-managed files scale from scripts to servers
>* Prevent leaks, clarify file lifetimes, handle errors



In [ ]:
#@title Python Code - Safely Managing Files with open() and with Blocks

# Demonstrate safe file handling using with blocks.
# Compare manual close handling with automatic context management cleanup.
# Show that files close even when exceptions occur.

import os

# Define a helper function that prints whether a path is open.
# This uses try opening for append mode to detect locking issues.
def print_file_status(path_label, path_name):
    try:
        with open(path_name, "a") as temp_file:
            print(path_label, "appears available for writing now.")
    except OSError as error:
        print(path_label, "seems locked or unavailable currently.")


# First, demonstrate manual open without using a with block.
# Forgetting close here can leave file descriptors open accidentally.
file_path_manual = "manual_example_log.txt"
manual_file = open(file_path_manual, "w")
manual_file.write("Manual open without using context manager.\n")
print("Manual file opened and written without automatic cleanup.")


# Intentionally forget to close manual_file here to simulate a bug.
# In real programs this can slowly exhaust operating system resources.

# Now, demonstrate safe usage with a with block context manager.
# The file will always close when the block finishes or errors.
file_path_safe = "safe_example_log.txt"
with open(file_path_safe, "w") as safe_file:
    safe_file.write("Safe open using with context manager.\n")
    print("Safe file opened and written inside with block.")


# Demonstrate that the safe file is closed after leaving the with block.
# Attempt another write using a new with block to confirm availability.
with open(file_path_safe, "a") as safe_file_append:
    safe_file_append.write("Second line appended safely after closing.\n")
    print("Safe file reopened and appended successfully after automatic close.")


# Show that even when an exception occurs, with still closes the file.
# The exception is caught so the script continues running normally.
file_path_error = "error_example_log.txt"
try:
    with open(file_path_error, "w") as error_file:
        error_file.write("Writing before raising an intentional error.\n")
        print("About to raise an intentional error inside with block.")
        raise ValueError("Simulated processing problem inside with block.")
except ValueError as caught_error:
    print("Caught error, but file from with block is already closed.")


# Check availability of each file path after operations complete.
# This demonstrates that with managed files are safely reusable immediately.
print_file_status("Manual file path", file_path_manual)
print_file_status("Safe file path", file_path_safe)
print_file_status("Error file path", file_path_error)


# Clean up created files so repeated runs stay tidy and predictable.
for cleanup_path in (file_path_manual, file_path_safe, file_path_error):
    if os.path.exists(cleanup_path):
        os.remove(cleanup_path)
print("Cleanup finished, temporary example files removed successfully.")



### **2.2. Standard Library Wrappers Around Built-ins as Context Managers**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_02_02.jpg?v=1765169900" width="250">



>* Standard wrappers turn low-level resources into context managers
>* They ensure automatic cleanup and reduce mental overhead

>* Context managers wrap low-level operations for safety
>* They ensure state is restored even after errors

>* Combine context managers to manage multiple resources together
>* Code stays shorter, clearer, and reliably cleaned up



In [ ]:
#@title Python Code - Standard Library Wrappers Around Built-ins as Context Managers

# Demonstrate standard library context manager wrappers around built-in style operations.
# Show automatic cleanup for locks and temporary standard output redirection.
# Highlight how wrappers simplify safe resource management and error handling.

import threading
import contextlib
import io

lock = threading.Lock()
fake_output = io.StringIO()

print("Before with blocks, lock is initially unlocked.")
print(f"Lock locked state before with: {lock.locked()}")

with lock:
    print(f"Inside lock block, lock locked state: {lock.locked()}")

print(f"After lock block, lock locked state: {lock.locked()}")

with contextlib.redirect_stdout(fake_output):
    print("This line goes into fake_output, not real console.")
    print("Context manager will restore original standard output automatically.")

print("After redirect, standard output is restored correctly.")
print("Captured redirected text follows on this line:")
print(fake_output.getvalue().strip())



### **2.3. Exception Flow and Automatic Cleanup in with Blocks**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_02_03.jpg?v=1765169947" width="250">



>* Exceptions trigger the context manager’s cleanup automatically
>* Files, locks, connections are always safely released

>* with blocks clean up resources after exceptions
>* separates cleanup from error handling, avoiding deadlocks

>* All exits from with blocks trigger cleanup
>* Ensures reliable resource handling in complex flows



In [ ]:
#@title Python Code - Exception Flow and Automatic Cleanup in with Blocks

# Demonstrate automatic cleanup when exceptions occur inside with blocks.
# Show that files close and locks release even after unexpected errors.
# Compare behavior with and without using with context managers.

import threading
import time

# Helper function that writes then raises, inside a with block.
def write_with_context(filename):
    print("Starting write_with_context, file will be opened.")
    with open(filename, "w") as f:
        print("Inside with block, writing first line.")
        f.write("First line before error.\n")
        print("Now raising an exception intentionally.")
        raise ValueError("Simulated problem while writing.")


# Helper function that writes then raises, without a with block.
def write_without_context(filename):
    print("Starting write_without_context, file will be opened.")
    f = open(filename, "w")
    print("Inside manual handling, writing first line.")
    f.write("First line before error.\n")
    print("Now raising an exception intentionally.")
    raise ValueError("Simulated problem while writing.")


# Demonstrate file cleanup behavior with try around the caller.
def demo_file_cleanup():
    print("\n--- File cleanup with with block demonstration ---")
    try:
        write_with_context("example_with.txt")
    except Exception as exc:
        print("Caught exception from with block:", type(exc).__name__)
    print("File example_with.txt is now safely closed.")


# Demonstrate missing automatic close when not using with block.
def demo_file_leak():
    print("\n--- File handling without with block demonstration ---")
    try:
        write_without_context("example_without.txt")
    except Exception as exc:
        print("Caught exception from manual handling:", type(exc).__name__)
    print("File example_without.txt may remain open until program ends.")


# Demonstrate lock cleanup when exceptions occur inside with block.
def demo_lock_cleanup():
    print("\n--- Lock cleanup with with block demonstration ---")
    lock = threading.Lock()

    def worker_with_lock():
        print("Worker acquiring lock using with block.")
        try:
            with lock:
                print("Worker inside critical section, raising error now.")
                raise RuntimeError("Simulated worker failure.")
        except RuntimeError as exc:
            print("Worker caught its own error:", type(exc).__name__)
        print("Worker finished, lock automatically released.")

    thread = threading.Thread(target=worker_with_lock)
    thread.start()
    thread.join()
    print("Main thread can acquire lock again successfully.")


demo_file_cleanup()

demo_file_leak()

demo_lock_cleanup()



## **3. Designing and Testing Simple Custom Context Managers**

### **3.1. Minimal Logging Context Manager Class**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_03_01.jpg?v=1765170009" width="250">



>* Context manager logs operation start and finish
>* Provides clean observability without cluttering core logic

>* Context manager logs standardized start and finish messages
>* Lets code focus on work while tracking operations

>* Logs failures and error details during exceptions
>* Still lets normal exception handling work correctly



In [ ]:
#@title Python Code - Minimal Logging Context Manager Class

# Demonstrate minimal logging context manager usage with simple tasks.
# Show how entry and exit messages appear around a with block.
# Show how exceptions are logged while still propagating normally.

import time
from datetime import datetime

class LoggingContext:
    def __init__(self, operation_name):
        self.operation_name = operation_name

    def __enter__(self):
        self.start_time = time.time()
        print(f"[START] {self.operation_name} at {datetime.now()}")
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        duration_seconds = time.time() - self.start_time
        if exc_type is None:
            print(f"[END] {self.operation_name} succeeded after {duration_seconds:.2f} seconds")
        else:
            print(f"[FAIL] {self.operation_name} failed after {duration_seconds:.2f} seconds")
            print(f"       Error type {exc_type.__name__} with message {exc_value}")
        return False

print("Running successful operation with logging context manager.")
with LoggingContext("Downloading sample report") as logger:
    time.sleep(0.5)
    print("Inside block, pretending to download report.")

print("\nRunning failing operation with logging context manager.")
try:
    with LoggingContext("Processing customer invoices") as logger:
        time.sleep(0.3)
        print("Inside block, about to raise error.")
        raise ValueError("Invoice total exceeded allowed limit")
except ValueError as error:
    print(f"Outside block, caught propagated error: {error}")



### **3.2. Nesting Custom Context Managers with Files and Locks**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_03_02.jpg?v=1765170063" width="250">



>* Each context handles its own setup, cleanup
>* with enters left-to-right, unwinds in reverse

>* Limit each context manager’s setup and cleanup responsibilities
>* Restore shared state; test layers independently with mocks

>* Nested contexts manage complex workflows and resources
>* Predictable unwinding ensures cleanup, metrics, and robustness



In [ ]:
#@title Python Code - Nesting Custom Context Managers with Files and Locks

# Demonstrates nesting custom context managers with files and locks together safely.
# Shows how each context manager handles its own setup and cleanup responsibilities.
# Prints messages that reveal the enter and exit ordering for all nested contexts.

import threading
import time
from contextlib import ContextDecorator


class TimingLogger(ContextDecorator):
    """Simple timing logger context manager for demonstration purposes only."""

    def __init__(self, label, log_file):
        self.label = label
        self.log_file = log_file
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time()
        print(f"[enter] timing logger started for {self.label}.")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        elapsed = time.time() - self.start_time
        status = "success" if exc_type is None else "error"
        message = f"{self.label} finished with {status} after {elapsed:.3f} seconds.\n"
        self.log_file.write(message)
        print(f"[exit] timing logger wrote message with status {status}.")
        return False


lock = threading.Lock()


def process_task(task_name, filename):
    print(f"Starting processing for task {task_name} now.")
    with open(filename, "a", encoding="utf-8") as log_file, lock:
        print("File opened and lock acquired successfully.")
        with TimingLogger(task_name, log_file):
            print("Inside timing logger context block now.")
            time.sleep(0.2)
            print("Task processing finished inside nested contexts.")
    print("File closed and lock released automatically.")


process_task("example_task", "example_log.txt")



### **3.3. Deliberately Raising Exceptions to Verify Context Manager Cleanup**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_07/Lecture_B/image_03_03.jpg?v=1765170114" width="250">



>* Test context managers under intentional error conditions
>* Ensure cleanup still runs and releases resources

>* Imagine realistic failures for your context manager
>* Raise exceptions and confirm resources are safely cleaned

>* Test nested contexts by triggering intentional errors
>* Confirm all managers clean up correctly and safely



In [ ]:
#@title Python Code - Deliberately Raising Exceptions to Verify Context Manager Cleanup

# Demonstrate custom context manager cleanup during exceptions.
# Show how __enter__ and __exit__ handle failures safely.
# Print messages proving cleanup happens even when errors occur.

class DemoResource:
    def __init__(self, name_label):
        self.name_label = name_label
        self.active_flag = False

    def __enter__(self):
        self.active_flag = True
        print(f"Entering context, resource {self.name_label} acquired.")
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        print(f"Exiting context, resource {self.name_label} cleanup starting.")
        self.active_flag = False
        if exc_type is not None:
            print(f"Exception noticed, type {exc_type.__name__} during block.")
        print(f"Resource {self.name_label} active flag now {self.active_flag}.")
        return False

print("Running normal block without raised exception.")
with DemoResource("test_one") as resource_one:
    print("Inside block, resource active flag is", resource_one.active_flag)

print("Running block that deliberately raises exception.")
try:
    with DemoResource("test_two") as resource_two:
        print("Inside failing block, resource active flag", resource_two.active_flag)
        raise ValueError("Simulated failure inside managed block.")
except ValueError as caught_error:
    print("Caught ValueError outside with block, message:", caught_error)



# <font color="#418FDE" size="6.5" uppercase>**Context Management and Cleanup: contextlib-related Patterns and with Protocol**</font>


In this lecture, you learned to:
- Explain how the with statement uses the context management protocol with built-in types in Python 3.12. 
- Use built-in types such as files and locks within with blocks to guarantee cleanup. 
- Design simple context-managed utilities that integrate cleanly with built-in behavior. 

In the next Module (Module 8), we will go over 'Metaprogramming, Classes, and Advanced Built-ins'